In [1]:
# IMPORTS
import requests

import gtfs_realtime_NYCT_pb2
import gtfs_realtime_pb2
import polars as pl
from polars import col
import re
from PIL import Image
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.graph_objects as go
import pandas as pd
from PIL import Image
import pyarrow
import json
import bisect
import math
from datetime import datetime, timedelta
import plotly.io as pio
import time

## Constants

In [3]:
STOP_STATUS = {"0": "Incoming At", "1": "Stopped At", "2": "In Transit To"}

In [4]:
API_ENDPOINTS = {
    "ACE": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-ace",
    "BDFM": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-bdfm",
    "G": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-g",
    "JZ": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-jz",
    "NQRW": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-nqrw",
    "L": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-l",
    "1234567": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs",
    "SI": r"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-si",
}

## Functions

In [7]:
def plot_map(
    coordinates,
    fig,
    trip_id,
    line,
    # hovertext,
    marker_size=15,
    marker_color="#03fca9",
):

    fig.add_trace(
        go.Scattermapbox(
            mode="markers+text",
            name=trip_id,
            lon=[coordinates[0]],
            lat=[coordinates[1]],
            text=line,
            textfont=dict(color="#ffffff"),
            marker={
                "size": marker_size,
                "color": color_lookup[line],
                # "symbol": "triangle-up",
            },
            showlegend=False,
            hoverinfo="text",
            hovertext=f"<b>Line {line}<b><br>{coordinates}",
        )
    )
    # fig.add_annotation(
    #     go.layout.Annotation(
    #         text=trip_id,
    #         showarrow=False,
    #         x=coordinates[0],
    #         y=coordinates[1],
    #         xref="x",
    #         yref="y",
    #         font=dict(size=12, color="black"),
    #         xanchor="center",
    #         yanchor="middle",
    #     )
    # )

# File Cleaning

# Data Capturing

## Master train info

## Testing

# Plotting Train

## Stop plotting

# Main Runtime

In [28]:
# if __name__ == "__main__":
#     app = dash.Dash(__name__)
#     with open("map_plot.json", "r") as f:
#         fig_json = json.load(f)
# 
#     fig = go.Figure(fig_json)
# 
#     @app.callback(
#         Output("live-map", "figure"),
#         [Input("interval-component", "n_intervals")],
#         # [State("live-map", "figure")],
#     )
#     def update_map(n):
#         fig = go.Figure(fig_json)
#         initialize_train_table(trains_tracked, last_updated, problems_log)
#         print("Initialize Done")
#         plot_trains(fig, trains_tracked)
#         stop_info_plotting(fig, trains_tracked, stop_schedule)
#         return fig
# 
#     app.layout = html.Div(
#         [
#             dcc.Graph(
#                 id="live-map",
#                 figure=fig,
#                 style={"width": "50vw", "height": "50vw"},
#             ),
#             dcc.Interval(id="interval-component", interval=60 * 1000, n_intervals=0),
#         ]
#     )
#     # app.run_server(debug=True)
#     trains_tracked = {}
#     problems_log = {}
#     stop_schedule = {}
#     stop_schedule = {stop_id: [] for stop_id in stops_clean["stop_id"].to_list()}
#     last_updated = {"last_updated": None}
# 
#     # Load the plot from an HTML file
#     # with open("map_plot.json", "r") as f:
#     #     fig_json = json.load(f)
# 
#     app.run_server(debug=True)
#     # fig = go.Figure(fig_json)
#     # 
#     # fig = go.Figure(fig_json)
#     # print("Starting")
#     # initialize_train_table(trains_tracked, last_updated, problems_log)
#     # print("Initialize Done")
#     # plot_trains(fig, trains_tracked)
#     # print("Train plotting done")
#     # stop_info_plotting(fig, trains_tracked, stop_schedule)
#     # print("Stop info Done")

In [29]:
raise Exception()

Exception: 

In [ ]:
# Function to generate the initial figure
def load_initial_figure(filename):
    with open(filename, "r") as f:
        fig_json = json.load(f)
    return go.Figure(fig_json)

# Callback function to update the map
def update_map_callback(n, fig_json, trains_tracked, last_updated, problems_log, stop_schedule):
    fig = go.Figure(fig_json)
    initialize_train_table(trains_tracked, last_updated, problems_log)
    print("Initialize Done")
    plot_trains(fig, trains_tracked)
    stop_info_plotting(fig, trains_tracked, stop_schedule)
    return fig

# Main function to run the app
def main():
    app = dash.Dash(__name__)

    # Load initial figure
    fig_json = load_initial_figure("map_plot.json")
    fig = go.Figure(fig_json)

    # Define layout
    app.layout = html.Div(
        [
            dcc.Graph(
                id="live-map",
                figure=fig,
                style={"width": "50vw", "height": "50vw"},
            ),
            dcc.Interval(id="interval-component", interval=60 * 1000, n_intervals=0),
        ]
    )

    # Initialize state variables
    trains_tracked = {}
    problems_log = {}
    stop_schedule = {stop_id: [] for stop_id in stops_clean["stop_id"].to_list()}
    last_updated = {"last_updated": None}

    # Define callback
    @app.callback(
        Output("live-map", "figure"),
        [Input("interval-component", "n_intervals")]
    )
    def update_map(n):
        return update_map_callback(n, fig_json, trains_tracked, last_updated, problems_log, stop_schedule)

    # Run the app
    app.run_server(debug=True)

In [ ]:
if __name__ == "__main__":
    main()

In [30]:
trains_tracked = {}
problems_log = {}
stop_schedule = {stop_id: [] for stop_id in stops_clean["stop_id"].to_list()}
last_updated = {"last_updated": None}


In [31]:
trains_tracked = initialize_train_table(trains_tracked, last_updated, problems_log)
print("Initialize Done")
stop_schedule = stop_schedule_creation(trains_tracked, stop_schedule)


Initialize Done


In [34]:
trains_tracked['080000_2..N01X175']

{'prev_departure_time': 1722710528,
 'prev_departure_station': '214',
 'planned_next_station': '213',
 'current_station': '214',
 'current_schedule': {'214N': {'arrival': 1722710558, 'departure': 1722710558},
  '213N': {'arrival': 1722710648, 'departure': 1722710648},
  '212N': {'arrival': 1722710768, 'departure': 1722710768},
  '211N': {'arrival': 1722710888, 'departure': 1722710888},
  '210N': {'arrival': 1722711008, 'departure': 1722711008},
  '209N': {'arrival': 1722711068, 'departure': 1722711068},
  '208N': {'arrival': 1722711158, 'departure': 1722711158},
  '207N': {'arrival': 1722711248, 'departure': 1722711248},
  '206N': {'arrival': 1722711338, 'departure': 1722711338},
  '205N': {'arrival': 1722711398, 'departure': 1722711398},
  '204N': {'arrival': 1722711488, 'departure': 1722711788},
  '201N': {'arrival': 1722711908, 'departure': 0}},
 'current_status': 1,
 'current_timestamp': 1722710528,
 'current_direction': 'N',
 'line': '2'}

In [33]:
stop_schedule

{'101': [],
 '103': [{'direction': 'N', 'line': '1', 'arrival': 1722710803},
  {'direction': 'N', 'line': '1', 'arrival': 1722712387},
  {'direction': 'N', 'line': '1', 'arrival': 1722711598},
  {'direction': 'N', 'line': '1', 'arrival': 1722711290},
  {'direction': 'N', 'line': '1', 'arrival': 1722712619},
  {'direction': 'N', 'line': '1', 'arrival': 1722713499},
  {'direction': 'N', 'line': '1', 'arrival': 1722713032}],
 '104': [{'direction': 'N', 'line': '1', 'arrival': 1722710713},
  {'direction': 'N', 'line': '1', 'arrival': 1722712297},
  {'direction': 'N', 'line': '1', 'arrival': 1722711508},
  {'direction': 'N', 'line': '1', 'arrival': 1722711200},
  {'direction': 'N', 'line': '1', 'arrival': 1722712529},
  {'direction': 'N', 'line': '1', 'arrival': 1722713409},
  {'direction': 'N', 'line': '1', 'arrival': 1722712942}],
 '106': [{'direction': 'N', 'line': '1', 'arrival': 1722710623},
  {'direction': 'N', 'line': '1', 'arrival': 1722712207},
  {'direction': 'N', 'line': '1', 'ar